<a href="https://colab.research.google.com/github/viv-bad/pytorch-course/blob/master/04_pytorch_custom_datasets_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. PyTorch Custom Datasets Video Notebook

We've used some datasets with PyTorch before, but how do we get our own data into PyTorch?

One of the ways to do so is via: custom datasets.

## Domain libraries

Depending on what you're working on, vision, text, audio, recommendation etc you want to look into each PyTorch domain libraries for existing data loading functions and customisable data loading functions.



## 0. Importing PyTorch and setting up device agnostic code

In [ ]:
import torch
from torch import nn
torch.__version__

In [ ]:
# Set up device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 1. Get data

We will use the food 101 dataset from PyTorch. (101 classes of food, 1000 images per class (75% train 25% test)

We will use a smaller subset of this database (3 classes of food and 10% of images)

When starting out ML projects, start small then increase the scale when necessary.

This speeds up how fast you can experiment.



In [ ]:
import requests
import zipfile
from pathlib import Path

# Setup path to datafolder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare..
if image_path.is_dir():
  print(f"{image_path} directory already exists... skipping download")
else:
  print(f"{image_path} does not exist... creating one")
  image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak sushi data
with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading data...")
  f.write(request.content)

# Unzip
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping data...")
  zip_ref.extractall(image_path)


In [ ]:
## 2. Becoming one with the data (data preparation and exploration)
import os
def walk_through_dir(dir_path):
  """Walks through dir_path returning its contents"""
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
walk_through_dir(image_path)

In [ ]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

### 2.1 Visualise an image


Let's write some code to:

1. Get all of the miage paths
2. Pick random image path using Python's `random.choice()`
3. Get the image class name `pathlib.Path.parent.stem`
4. Since we are working with images, let's open them with Python's `PIL`
5. We'll then show the image and print metadata

In [ ]:
import random
from PIL import Image

# random.seed(42)

# 1. Get all image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))

# 2. Pick random image path
random_image_path = random.choice(image_path_list)

random_image_path

# 3. Get image class from path name (name of directory)
image_class = random_image_path.parent.stem
print(image_class)

# 4. Open Image
img = Image.open(random_image_path)

# 5. Print metadata
print(f"Random image path {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tuen the image into an array
img_as_array = np.asarray(img)

# Plot image
plt.figure(figsize=(10,7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height, width, colour_channels]")
plt.axis(False)

## 3. Transforming data

Before we can use our image data with PyTorch:

1. turn target data into tensors (a numerical representation of images)
2. Turn it into a `torch.utils.data.Dataset` and subsequently a `torch.utils.data.DataLoader` to create an interable/batched version of our data. These will be called `Dataset` and `DataLoader`.



In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


### 3.1 Transforming data with `torchvision.transforms`

Transforms help you get your images ready to be used with a model/perform data augmentation

In [ ]:
# Write a transform for image
data_transform = transforms.Compose([
    #Resize our images to 64x64 - allows use of tinyVG model
    transforms.Resize(size=(64,64)),
    #Flip images randomly on horizontal axis (increases diversity of dataset)
    transforms.RandomHorizontalFlip(p=0.5),
    # Turn image into torch tensor
    transforms.ToTensor()
])

In [ ]:
data_transform(img)

In [ ]:
def plot_transformed_images(image_paths, transform, n= 3, seed=None):
  """
  Selects random images from path of images and loads/transforms them, then plots the original vs the transformed version.
  """

  if seed:
    random.seed(seed)

  random_image_paths = random.sample(image_paths, k=n)
  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(1,2)
      ax[0].imshow(f)
      ax[0].set_title(f"Original\nSize: {f.size}")
      ax[0].axis(False)

      # Transform and plot target image
      transformed_f = transform(f).permute(1,2,0)
      ax[1].imshow(transformed_f)
      ax[1].set_title(f"Transformed\nShape: {transformed_f.shape}")
      ax[1].axis(False)

      fig.suptitle(f"Class {image_path.parent.stem}", fontsize=16)


plot_transformed_images(image_path_list, data_transform, n=3, seed=42)




## 4. Option 1: Loading image data using `ImageFolder`

We can load image classification data using `torchvision.datasets.ImageFolder`




In [ ]:
# Use Imagefolder to create dataset(s)

from torchvision import datasets

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform = None)

test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

train_data, test_data



In [ ]:
# Get class names as list
class_names = train_data.classes

In [ ]:
# Get class names as dict
class_dict = train_data.class_to_idx

In [ ]:
# Check lengths of our dataset
len(train_data), len(test_data)

In [ ]:
# Index on the train_data dataset to get a single image and label

img, label = train_data[0][0], train_data[0][1]
print(f"Image tensor:\n {img}")
print(f"Image shape:\n {img.shape}")
print(f"Image datatype:\n {img.dtype}")
print(f"Image label:\n {label}")
print(f"Image label datatype:\n {type(label)}")

In [ ]:
# Rearrange order of dimensions
img_permute= img.permute(1,2,0)

# Print out different shapes
print(f"Original shape: {img.shape} -> [colour_channels, height, width]")
print(f"Original shape: {img_permute.shape} -> [height, width, colour_channels]")

#Plot the image
plt.figure(figsize=(10,7))
plt.imshow(img_permute)
plt.axis("off")
plt.title(class_names[label], fontsize=14)

### 4.1 Turn loaded images into `DataLoader`'s

A `DataLoader` is going to help us turn our `Dataset`'s into iterables and we can customise the `batch_size` so our model can see `batch_size` images at a time.

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 1
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=1, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=1, shuffle=False)

train_dataloader, test_dataloader

In [ ]:
len(train_dataloader), len(test_dataloader)

In [ ]:
img, label = next(iter(train_dataloader))

# Batch size will now be 1, can change
print(f"Image shape: {img.shape} ->. [batch_size, color channels, height, width]")
print(f" Label shape: {label.shape}")

## 5. Option 2: Loading ImageData with a Custom Dataset

1. Want to be able to load images from file
2. Want to be able to get class names from Dataset
3. Want to get classes as dictionary from Dataset

Pros:
* Can create a `Dataset` out of almost anything
* Not limited to PyTorch pre-built `Dataset` functions

Cons:
* Even though you could create `Dataset` out of almost anything, it doesn't mean it will work
* Using a custom `Dataset` requires more code, which could be prone to errors or performance issues





In [ ]:
import os
import pathlib
import torch

from PIL import Image
from torch.utils.data import Dataset # Base Dataset class
from torchvision import transforms
from typing import Tuple, Dict, List



In [ ]:
# Instrance of torchvision.datasets.Imagefolder()

train_data.classes, train_data.class_to_idx

In [ ]:
# We want a helper function to return the data like above ^



### 5.1 Creating a helper function to get class names

We want a function to:

1. Get the class names using `os.scandir()` to traverse a target dir and the dir is in standard image classification format (train/test, images in class dirs).
2. Raise an error if the class names aren't found. (i.e. something wrong with dir structure)
3. Turn class names into a dict and list and return them



In [ ]:
# Setup path for target directory
target_directory = train_dir
print(f"Target dir: {target_directory}")

# Get the class names from the target directory
class_names_found = sorted([entry.name for entry in list(os.scandir(target_directory))])
class_names_found

In [ ]:
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
  """Given a target directory, finds the class folder names"""
  # 1. Get class names by scanning target directory
  classes = sorted([entry.name for entry in list(os.scandir(directory)) if entry.is_dir])

  # 2. Raise error if no classes found
  if not classes:
    raise FileNotFoundError(f"Couldn't find any classes in {directory}")

  # 3. Turn class names to dict idx of index labels
  class_to_idx = {class_name: i for i, class_name in enumerate(classes)}
  return classes, class_to_idx


In [ ]:
find_classes(target_directory)

### 5.2 Create a custom `Dataset` to replicate `ImageFolder`

To create our own custom dataset, we want toÑ


1. Subclass `torch.utils.data.Dataset`
2. Init our subclass with a target directory (the directory we want to get data from), as well as a transform if we want to transform our data.
3. Create several attributes:
  * paths - paths of our images
  * transform - transform we want to use
  * classes - a list of target classes
  * class_to_idx - dict of target classes mapped to integer labels
4. Create a function to `load_images()`, this function will open an image
5. Overwrite `__len()__` method to return the length of the dataset
6. Overwrite the `__getitem()__` method to return a given sample when passed an index



In [ ]:
# Write a custom dataset class
from torch.utils.data import Dataset

# 1. Subclass torch.utils.data.Dataset
class ImageFolderCustom(Dataset):
  # 2. Init custom dataset
  def __init__(self, targ_dir: str, transform=None):
    # 3. Create class attributes
    # Get all image paths
    self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))
    # Setup transforms
    self.transforms = transform
    # Create classes and class_to_idx attrs
    self.classes, self.class_to_idx = find_classes(targ_dir)

  # 4. Load images function
  def load_image(self, index: int) -> Image.Image:
    """Opens an image via a path and returns it"""
    image_path = self.paths[index]
    return Image.open(image_path)

  # 5. Overwrite __len()__
  def __len__(self) -> int:
    """Returns the total number of samples"""
    return len(self.paths)

  # 6. Overwrite __getitem()__
  def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
    """Returns one sample of data, data and label (X, y)."""
    img = self.load_image(index)
    class_name = self.paths[index].parent.name #expects path in format data_folder/class_name/image.jpg
    class_idx = self.class_to_idx[class_name]

    # Transform if necessary
    if self.transforms:
      return self.transforms(img), class_idx # return data, label (X, y)
    else:
      return img, class_idx # return untransformed image and label




In [ ]:
# Create a transform
from torchvision import transforms

train_transforms = transforms.Compose([transforms.Resize(size=(64,64)),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.Resize(size=(64,64)),
                                       transforms.ToTensor()])




In [ ]:
# test out image folder custom
train_data_custom = ImageFolderCustom(targ_dir = train_dir,
                                      transform = train_transforms,
                                      )
test_data_custom = ImageFolderCustom(targ_dir = test_dir,
                                      transform = test_transforms,
                                      )




In [ ]:
train_data_custom, test_data_custom

In [ ]:
len(train_data), len(train_data_custom)

In [ ]:
len(test_data), len(test_data_custom)

In [ ]:
train_data_custom.classes

In [ ]:
train_data_custom.class_to_idx

In [ ]:
# Check for equality between original ImageFolder and ImageFolderCustom datasets
print(train_data_custom.classes == train_data.classes)
print(test_data_custom.classes == test_data.classes)

### 5.3 Create a function to display random images

1. Take in `Dataset` and a number of other params like class names and how many images to visualise
2. To prevent the display getting too big, cap to 10
3. Set the random seed for reprod
4. Get a list of random sample indexes from the target dataset.
5. Setup a matplotlib plot.
6. Loop through the random sample idxs and plot them with matplotlib.
7. Make sure the dimensions of our images line up with matplotlin (HWC)

In [ ]:
# 1. Create a function to take in a dataset
def display_random_images(dataset: torch.utils.data.Dataset, classes: List[str] = None, n: int = 10, display_shape: bool = True, seed: int = None):
  # 2. Adjust display if n too high
  if n > 10:
    n = 10
    display_shape = False
    print(f"For display, n should be <=10, setting to 10 and remove shape display")

  # 3. set seed
  if seed:
    random.seed(seed)

  # 4. Get random sample idxs
  random_samples_idx = random.sample(range(len(dataset)), k = n)

  # 5. set up plot
  plt.figure(figsize=(16,8))

  # 6. Loop through random sample idxs and plot them
  for i, targ_sample in enumerate(random_samples_idx):
    targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

    # 7. Adjust tensor dimensions for plotting
    targ_image_adjust = targ_image.permute(1,2,0) # [C,H,W] -> [H,W,C]

    # Plot adjusted sampples
    plt.subplot(1, n, i+1)
    plt.imshow(targ_image_adjust)
    plt.axis(False)
    if classes:
      title = f"Class: {classes[targ_label]}"
      if display_shape:
        title = title + f"\nshape: {targ_image_adjust.shape}"
    plt.title(title)


In [ ]:
display_random_images(train_data, n=5,classes=class_names, seed=None)

In [ ]:
display_random_images(train_data_custom, n=5,classes=class_names, seed=None)

### 5.4 Turn custom loaded images in to `DataLoader`'s

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_dataloader_custom = DataLoader(dataset=train_data_custom, batch_size=BATCH_SIZE, num_workers=0, shuffle=True)
test_dataloader_custom = DataLoader(dataset=test_data_custom, batch_size=BATCH_SIZE, num_workers=0, shuffle=False)

train_dataloader_custom, test_dataloader_custom

In [ ]:
# Get image and label from custom data loader
img_custom, label_custom = next(iter(train_dataloader_custom))

img_custom.shape, label_custom.shape

## 6. Other forms of transforms (data augmentation)

Data augmentation is the process of artificially adding diversity to your training data.

In the case of image data, this may mean applying various image transformations to the training images.

This will result in a more generalizable model to unseen data.


Let's take a look at one particular type of data aug used to train pytroch vision models to state of the art levels.

In [ ]:
# Let's look at trivial augment...
from torchvision import transforms

train_transform = transforms.Compose([
                                      transforms.Resize(size=(224,224)),
                                      transforms.TrivialAugmentWide(num_magnitude_bins=32), #intensity of augmentation
                                      transforms.ToTensor()
                                      ])


test_transform = transforms.Compose([
                                      transforms.Resize(size=(224,224)),
                                      transforms.ToTensor()
                                      ])




In [ ]:
# GEt all image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))
image_path_list[:10]

In [ ]:
plot_transformed_images(
    image_paths=image_path_list, transform=train_transform, n = 3, seed = None
)

## 7. Model 0: TinyVGG without data augmentation


### 7.1 Creating transforms and loading data for Model 0

In [ ]:
# Create simple transform
simple_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])



In [ ]:
# 1. Load and transform data
from torchvision import datasets
train_data_simple = datasets.ImageFolder(root=train_dir, transform=simple_transform)
test_data_simple = datasets.ImageFolder(root=test_dir, transform=simple_transform)

# 2. turn datasets into dataloaders
import os
from torch.utils.data import DataLoader

# Setup batchsize and num workers
BATCH_SIZE=32
NUM_WORKERS = os.cpu_count()

# Create dataloaders
train_dataloader_simple = DataLoader(dataset=train_data_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers = NUM_WORKERS)

test_dataloader_simple = DataLoader(dataset=test_data_simple, batch_size=BATCH_SIZE, shuffle=False, num_workers = NUM_WORKERS)


### 7.2 Create TinyVGG model class

In [ ]:
class TinyVGG(nn.Module):
  """Model architecture copying TinyVGG from CNN Explainer"""
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2) # default stride is same as kernel size
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2) # default stride is same as kernel size
    )

    self.classifier = nn.Sequential(
        nn.Flatten(), # turns convolutional blocks into feature vectors
        nn.Linear(in_features=hidden_units * 13 * 13 # this makes the matrices mat1 and mat2 line up after flattening
                  , out_features=output_shape)

    )

  def forward(self, x):
    x = self.conv_block_1(x)
    # print(x.shape)
    x = self.conv_block_2(x)
    # print(x.shape)
    x = self.classifier(x)
    # print(x.shape)
    return x

    # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # speed up GPU computations with operator fusion instead

In [ ]:
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3,# number of color channels in out image data
                  hidden_units=10,
                  output_shape = len(class_names)

                  ).to(device)

model_0

### 7.3 Try a forward pass on a single image to test the model

In [ ]:
# Get a single image batch
image_batch, label_batch = next(iter(train_dataloader_simple))
image_batch.shape, label_batch.shape

In [ ]:
# Try a forward pass
model_0(image_batch.to(device))

### 7.4 use `torchinfo` to get an idea of the shapes going through our model

In [ ]:
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

from torchinfo import summary
summary(model_0, input_size=[1,3,64,64]) # example input of a batch of one image [batch_size, colour_channel, height, width]
# Summary here mocks a forward pass in the model with the input size you give it


## 7.5 Create train and test loop functions

* `train_step()` takes in model and dataloader and trains model on dataloader.
* `test_step()` takes in a model and dataloader and evaluates the model on the dataloader.

In [ ]:
# Create train_step()

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device=device):
  # Put model in train mode
  model.train()

  # Set up eval metrics
  train_loss, train_acc = 0,0

  # loop through data loader data batches
  for batch, (X,y) in enumerate(dataloader):
    # send data to target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X) # outputs model logits

    # 2. Calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. optimiser 0 grad
    optimizer.zero_grad()

    # 4. loss backwards
    loss.backward()

    # 5. optimizer step
    optimizer.step()

    # calculate the accuracy metric
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)

    train_acc += (y_pred_class==y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)

  return train_loss, train_acc

In [ ]:
# Create test_step()
def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, device=device):

  # put model in eval mode
  model.eval()

  # set up test loss and test acc
  test_loss, test_acc = 0,0

  # Tuyrn on inference mode
  with torch.inference_mode():
    # loop through dataloader batches
    for batch, (X, y) in enumerate(dataloader):
      # send data to the target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Calculate loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # 3. Calculate accuracy
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/ len(test_pred_labels))
    # Adjust metrics to get av loss and acc per batch
    test_loss = test_loss/ len(dataloader)
    test_acc = test_acc / len(dataloader)

    return test_loss, test_acc

### 7.6 Creating a `train()` function to combine `train_step()` and `test_step()`

In [ ]:
from tqdm.auto import tqdm
#1. Create train function that takes in various model params etc...
def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, optimizer: torch.optim.Optimizer, loss_fn: torch.nn.Module = nn.CrossEntropyLoss(), epochs: int = 5, device=device):
  """Train model"""

  # 2. Create empty results dict
  results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model, dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer, device=device)
    test_loss, test_acc = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device)

    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

    # 5. Update results dict
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return stored results
  return results

### 7.7 Train and evaluate model 0


In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = TinyVGG(input_shape=3, # number of colour channels in images
                  hidden_units = 10,
                  output_shape = len(train_data.classes)
                  )

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_0.parameters(), lr = 0.001)

#timers
from timeit import default_timer as timer

start_time = timer()


# train model_0
model_0_results = train(model=model_0, train_dataloader = train_dataloader_simple, test_dataloader = test_dataloader_simple, optimizer = optimizer, loss_fn = loss_fn, epochs = NUM_EPOCHS, device=device)

# End timer
end_time = timer()
print(f"Total training time: {end_time - start_time:.3f} seconds")

In [ ]:
model_0_results

### 7.8 Plot the loss curvers of Model 0

A **loss curve** is a way of trakcing your model's performance over time


In [ ]:
# Get model_0_results_keys
model_0_results.keys()

In [ ]:
def plot_loss_curves(results: Dict[str, List[float]]):
  """Plots training curves of a results dictionary"""
  # Get loss vaules of the results dictionary (training and test)
  loss = results["train_loss"]
  test_loss = results["test_loss"]


  # Get accuracy
  accuracy = results["train_acc"]
  test_accuracy = results["test_acc"]

  # num epochs
  epochs = range(len(results["train_loss"])
  )

  # setup plit
  plt.figure(figsize=(15,7))

  # plto loss
  plt.subplot(1,2,1)
  plt.plot(epochs, loss, label="train_loss")
  plt.plot(epochs, test_loss, label="test_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  #Plot accuracy

  plt.subplot(1,2,2)
  plt.plot(epochs, accuracy, label="train_accuracy")
  plt.plot(epochs, test_accuracy, label="test_accuracy")
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend()



In [ ]:
plot_loss_curves(model_0_results)

## 8. What should an ideal loss curve look like?

A loss curve is one of the best ways to troubleshoot a model, we want it to go down overtime, while accuracy goes up over time.

https://developers.google.com/machine-learning/crash-course/overfitting/interpreting-loss-curves

## 9. Model 1: TinyVGG with data augmentation - helps deal with overfitting (augmentation)

- Artificially increases diversity of dataset without getting more data

### 9.1 Create transform with data augmentation


In [ ]:
# Create training transform
from torchvision import transforms
train_transform_trivial = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

test_transform_simple = transforms.Compose([ # never augment the test dataset
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])




### 9.2 Create train and test `Dataset`'s and `DataLoader`'s with data augmentation

In [ ]:
# Turn image folders into Datasets
from torchvision import datasets
train_data_augmented = datasets.ImageFolder(root=train_dir, transform=train_transform_trivial)

test_data_simple = datasets.ImageFolder(root=test_dir, transform=test_transform_simple)




In [ ]:
# Turn datasets into dataloaders
import os
from torch.utils.data import DataLoader
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

torch.manual_seed(42)

train_dataloader_augmented = DataLoader(dataset=train_data_augmented, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader_simple = DataLoader(dataset=test_data_simple, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


### 9.3 Construct and train model 1

This time we will use the same model architecture, but this time we have augmented the training dataset.

Remember, you typically start as simple as possible then iterate and add complexity to improve model performance.

In [ ]:
# Create model_1 and send it to the target device
torch.manual_seed(42)
model_1 = TinyVGG(input_shape=3, hidden_units=10, output_shape= len(train_data_augmented.classes)).to(device)
model_1

Now we have a model and dataloaders, let's make the loss function and optimizer and call `train()` to train and evaluate our model

In [ ]:
# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 5

# Setup loss and optim
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_1.parameters(), lr=0.001)

# Start timer
from timeit import default_timer as timer
start_time = timer()

# train model 1
model_1_results = train(model=model_1, train_dataloader=train_dataloader_augmented, test_dataloader=test_dataloader_simple, optimizer=optimizer, loss_fn=loss_fn, epochs=NUM_EPOCHS, device=device)

end_time = timer()
print(f"Total training time for model_1: {end_time-start_time:.3f} seconds")

### 9.4 Plot the loss curves od model_1

In [ ]:
model_1_results

In [ ]:
plot_loss_curves(model_1_results)

## 10. Compare model results

After evaluating our modelling experiments compare them

There's a few ways to do this:
1. Hard coding (what we're doing)
2. Tensorboard
3. Weights & Biases

In [ ]:
import pandas as pd
model_0_df = pd.DataFrame(model_0_results)
model_1_df = pd.DataFrame(model_1_results)

In [ ]:
# Plot model results on same plot
plt.figure(figsize=(15, 10))

epochs = range(len(model_0_df))

plt.subplot(2,2,1)
plt.plot(epochs, model_0_df["train_loss"], label="Model 0")
plt.plot(epochs, model_1_df["train_loss"], label="Model 1")
plt.title("Train Loss")
plt.xlabel("Epochs")
plt.legend()


plt.subplot(2,2,2)
plt.plot(epochs, model_0_df["test_loss"], label="Model 0")
plt.plot(epochs, model_1_df["test_loss"], label="Model 1")
plt.title("Test Loss")
plt.xlabel("Epochs")
plt.legend()

plt.subplot(2,2,3)
plt.plot(epochs, model_0_df["train_acc"], label="Model 0")
plt.plot(epochs, model_1_df["train_acc"], label="Model 1")
plt.title("Train Accuracy")
plt.xlabel("Epochs")
plt.legend()

plt.subplot(2,2,4)
plt.plot(epochs, model_0_df["test_acc"], label="Model 0")
plt.plot(epochs, model_1_df["test_acc"], label="Model 1")
plt.title("Test Accuracy")
plt.xlabel("Epochs")
plt.legend()

